In [ ]:
import pandas as pd

exo_traindf = pd.read_csv('https://student-datasets-bucket.s3.ap-south-1.amazonaws.com/whitehat-ds-datasets/kepler-exoplanets-dataset/exoTrain.csv')
exo_traindf.head()

In [ ]:
exo_testdf = pd.read_csv('https://student-datasets-bucket.s3.ap-south-1.amazonaws.com/whitehat-ds-datasets/kepler-exoplanets-dataset/exoTest.csv')
exo_testdf.head()

In [ ]:
exo_traindf['LABEL'].value_counts()

In [ ]:
exo_testdf['LABEL'].value_counts()

In [ ]:
print(exo_traindf.shape)
print(exo_testdf.shape)

In [ ]:

exo_traindf.isnull()

In [ ]:
exo_traindf.isnull().sum()

In [ ]:
# obtaining a list of columns in the dataset
exo_traindf.columns

In [ ]:
# fetching values of a column
exo_traindf['LABEL']

In [ ]:
# counting the missing values
num_missing_values = 0

for i in exo_traindf.columns:
    for j in exo_traindf[i].isnull():
        if j==True:
            num_missing_values += 1

print("Total missing values in the training dataset =", num_missing_values)

In [ ]:
# creating pandas series for first and last two stars of the dataset
star0 = exo_traindf.iloc[0, :]
star1 = exo_traindf.iloc[1, :]
star5085 = exo_traindf.iloc[5085, :]
star5086 = exo_traindf.iloc[5086, :]

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

plt.figure(figsize=(16,4))
x_values_star0 = np.arange(1,3198)
y_values_star0 = star0[1:] # we ignore LABEL column as it's not a flux value

sns.scatterplot(x=x_values_star0, y=y_values_star0)
plt.show()

In [ ]:
plt.figure(figsize=(16,4))
x_values_star0 = np.arange(1,3198)
y_values_star0 = star0[1:] # we ignore LABEL column as it's not a flux value

sns.lineplot(x=x_values_star0, y=y_values_star0)
plt.show()

In [ ]:
plt.figure(figsize=(16,4))
x_values_star1 = np.arange(1,3198)
y_values_star1 = star1[1:]

sns.scatterplot(x=x_values_star1, y=y_values_star1)
plt.show()

In [ ]:
plt.figure(figsize=(16,4))
sns.lineplot(x= x_values_star1,y= y_values_star1)
plt.show()

In [ ]:
plt.figure(figsize=(16,4))
x_values_star5085 = np.arange(1,3198)
y_values_star5085 = star5085[1:]

sns.scatterplot(x=x_values_star5085, y=y_values_star5085)
plt.show()

In [ ]:
plt.figure(figsize=(16,4))
x_values_star5085 = np.arange(1,3198)
y_values_star5085 = star5085[1:]

sns.lineplot(x=x_values_star5085, y=y_values_star5085)
plt.show()

In [ ]:
plt.figure(figsize=(16,4))
x_values_star5086 = np.arange(1,3198)
y_values_star5086 = star5086[1:]

sns.scatterplot(x=x_values_star5086, y=y_values_star5086)
plt.show()

In [ ]:
plt.figure(figsize=(16,4))
x_values_star5086 = np.arange(1,3198)
y_values_star5086 = star5086[1:]

sns.lineplot(x=x_values_star5086, y=y_values_star5086)
plt.show()

In [ ]:
exo_testdf['LABEL'].value_counts()

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# separating the feature variables (flux values) and target variable (Label)
feat_traindf = exo_traindf.iloc[:, 1:]
target_traindf = exo_traindf.iloc[:, 0]

# fitting the model with feature and target variables
model = RandomForestClassifier(n_jobs=-1, n_estimators=50)
model.fit(feat_traindf, target_traindf)
model.score(feat_traindf, target_traindf)

In [ ]:
feat_testdf = exo_testdf.iloc[:, 1:]
target_testdf = exo_testdf.iloc[:, 0]

# calling the predict() function to make predictions on the test dataset
predicted = model.predict(feat_testdf)
print(predicted)

In [ ]:
predicted_series = pd.Series(predicted)
predicted_series.value_counts()

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

confusion_matrix(target_testdf, predicted)

In [ ]:
print(classification_report(target_testdf, predicted))

In [ ]:
exo_traindf.describe()

In [ ]:
def mean_normalise(series):
    # vectorised operation
    norm = (series-series.mean()) / (series.max()-series.min())
    return norm

In [ ]:
# creating a dataframe with normalised flux values, leaving the LABEL column 
# we apply the function mean_normalise(), horizontally keeping the axis=1
norm_traindf = exo_traindf.iloc[:, 1:].apply(mean_normalise, axis=1)
# inserting the LABEL column in the norm_traindf 
norm_traindf.insert(loc=0, column='LABEL', value= exo_traindf['LABEL'])
norm_traindf.head()

In [ ]:
norm_testdf = exo_testdf.iloc[:, 1:].apply(mean_normalise, axis=1)
norm_testdf.insert(loc=0, column='LABEL', value=exo_testdf['LABEL'])
norm_testdf.head()

In [ ]:
# taking the transpose of the dataframe
exo_traindf.T

In [ ]:
# we applied abs() function because the transformed values sometimes turn out to be complex numbers
fft_star0 = np.abs(np.fft.fft(star0))

# frequency array will be same for all the stars because it depends on the length of series (all stars have same array length)
frequency = np.fft.fftfreq(len(star0))

plt.figure(figsize=(16,4))
x_value = frequency
y_value = fft_star0
plt.plot(x_value, y_value)
plt.show()


In [ ]:
def fft_func(series):
    fft_star = np.fft.fft(series, n=len(series))
    return np.abs(fft_star)

In [ ]:
traindf_T = norm_traindf.iloc[:, 1:].T
fft_traindf_T = traindf_T.apply(fft_func, axis = 0)
# After transforming the 'FLUX' values, we again take the transpose.
fft_train = fft_traindf_T.T
fft_train.head()

In [ ]:
# applying fft on test dataset
testdf_T = norm_testdf.iloc[:, 1:].T
fft_testdf_T = testdf_T.apply(fft_func, axis = 0)
# After transforming the 'FLUX' values, we again take the transpose.
fft_test = fft_testdf_T.T
fft_train.head()

In [ ]:
from imblearn.over_sampling import SMOTE

train = norm_traindf['LABEL']
test = norm_testdf['LABEL']

# sampling_strategy=1 implies that number of samples having label 1 and 2 should be equal
smote = SMOTE(sampling_strategy=1)
x_fft_train, y_fft_train = smote.fit_resample(fft_train,train)

print(x_fft_train.shape)
print(y_fft_train.shape)

In [ ]:
model = RandomForestClassifier(n_jobs=-1, n_estimators=50)
model.fit(x_fft_train, y_fft_train)

predicted = model.predict(fft_test)
print(predicted)

In [ ]:
import xgboost as xg
model = xg.XGBClassifier()

# replacing label values 1 with 0 and 2 with 1- to avoid error of invalid classes
y_fft_train = y_fft_train.replace({1: 0, 2: 1}) 
model.fit(x_fft_train, y_fft_train)

model_predict = model.predict(np.array(fft_test))
print(model_predict)

In [ ]:
confusion_matrix(test,model_predict)